<a href="https://colab.research.google.com/github/03-hub/ThinkStats2/blob/master/SemEvalNLPTask_LSTM%2BBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')


!git clone https://github.com/ncg-task/training-data.git "/content/train"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path '/content/train' already exists and is not an empty directory.


In [7]:
import os
import glob
import numpy as np
import pandas as pd
import sklearn
import torch
import tensorflow as tf
import random
!pip install transformers
from sklearn.model_selection import train_test_split
from transformers import BertConfig, BertTokenizerFast, TFBertForSequenceClassification

In [8]:
def Articlesandcontributions(path):
    articles = []
    contributions = []

    for task_name in os.listdir(path):  #/training-data
        if task_name != 'README.md' and task_name != '.git':
          article_category = os.path.join(path, task_name)  # ./training-data/natural_language_inference

          for folder_name in sorted(os.listdir(article_category)):
              article_index = os.path.join(article_category, folder_name)  # ./datasets/training-data/natural_language_inference/0

              with open(glob.glob(os.path.join(article_index, '*-Stanza-out.txt'))[0], encoding='utf-8') as f:#I used glob to retrieve files with matching pattern
                  article = f.read()                                                                           #* wild card matches the files with stanze out.txt
                  articles.append(article.lower())
                  
              with open(os.path.join(article_index, 'sentences.txt'), encoding='utf-8') as f:
                  contribution = []
                  for line in f.readlines():
                      article_contribution = int(line.strip())
                      contribution.append(article_contribution)
                  contributions.append(contribution) #contribution=[99,10.....]
          #     break
          # break
    return articles, contributions

In [11]:
def SentenceAndLables(articles, contributions):
    sentences = []
    labels = []
    for i, article in enumerate(articles):
        contribution = contributions[i]

        sents = article.split('\n')[0:-1]
        rand_sents=random.sample(sents,60)
        #random sampling.Choosing 60 random sentences in an article because the input is huge and this device is 
        #not supporting
        for j, sent in enumerate(rand_sents):
            sentences.append(sent)
            if (j + 1) in contribution:
                labels.append(1)
            else:
                labels.append(0)
    return sentences, labels

In [9]:
train_input_dir='/content/train'

In [12]:
train_articles, train_contributions = Articlesandcontributions(train_input_dir)

train_sentences, train_labels = SentenceAndLables(train_articles, train_contributions)
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_sentences, train_labels, test_size=.2)
#def train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)

#Split arrays or matrices into random train and test subsets.

#list: train_sentences
#(11376 items) ['abstract', 'for instance in the ...', 'sqs is a..]
#list: train_labels
#(11376 items) [0, 1, 0, 0, 0, ...]

#list: val_sentences
#(2844 items) ['the importance score...', 'this model encodes a...]
#list: val_labels
#(2844 items) [1, 0, 1, 0, 1, ...]
print(len(train_labels))
print(len(val_labels))

11376
2844


In [26]:

########### Library import and random seed
import numpy as np
from sklearn import metrics
import transformers
import torch
import random
from transformers import AutoTokenizer, AutoModel
seed_val = 66
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#### Hyper-parameters for the model
EPOCHS = 3
LEARNING_RATE = 1e-03
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')			
MAX_LEN = 100
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
import matplotlib.pyplot as plt
%matplotlib inline
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')


In [13]:


#### Create Dataset Loader
class Triage():
    def __init__(self, data_in,data_out, tokenizer, max_len):
        self.len = len(data_in)
        self.data = data_in
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label = data_out
        
    def __getitem__(self, index):
        # title = str(self.data.TITLE[index])
        # title = " ".join(title.split())
        title = self.data[index]
        inputs = self.tokenizer.encode_plus(title,
            None,add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True,
            return_length = True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']
        lengths = inputs['length']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'lengths': torch.tensor(lengths, dtype=torch.long),
            'targets': torch.tensor(self.label[index], dtype=torch.long)
        } 
    def __len__(self):
      return self.len
  

In [14]:

#### Train and Valid Loader 
from torch.utils.data import DataLoader
training_set = Triage(train_sentences,train_labels, tokenizer, MAX_LEN)
train_params = {'batch_size': TRAIN_BATCH_SIZE,'shuffle': True,'num_workers': 0}
training_loader = DataLoader(training_set, **train_params)
valid_set = Triage(val_sentences,val_labels, tokenizer, MAX_LEN)
valid_params = {'batch_size': VALID_BATCH_SIZE,'shuffle': True,'num_workers': 0}
valid_loader = DataLoader(valid_set, **valid_params)

In [3]:
from collections import Counter
from torch import cuda
import torch.nn.functional as F
device = 'cuda' if cuda.is_available() else 'cpu'

class SCIBERTClass(torch.nn.Module):
    def __init__(self):
        super(SCIBERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased',  output_hidden_states=True)
        self.lstm = torch.nn.LSTM(768, 400, num_layers=2, batch_first = True, bidirectional=True)
        self.l2 = torch.nn.Dropout(0.1)
        self.l3 = torch.nn.Linear( 800,400)
        self.l4=torch.nn.Linear(400,100)
        self.l5=torch.nn.Linear(100,2)

    
    def forward(self, ids, mask, token_type_ids, lengths):
        encoded_layers = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)[2]
        scibert_hidden_layer = encoded_layers[12]
        enc_hiddens, (last_hidden, last_cell) = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(scibert_hidden_layer, lengths, batch_first=True, enforce_sorted=False)) #enforce_sorted=False  #pack_padded_sequence(data and batch_sizes
        output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)  # (batch_size, 2*hidden_size)
        output_hidden = self.l2(output_hidden)            # no size change
        output_2 = self.l3(output_hidden)
        output_2 = torch.nn.ReLU()(output_2)
        output_4 = self.l4(output_2)
        output_5 = self.l5(output_4)
        return output_5

model = SCIBERTClass()
model.to(device)




Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SCIBERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [32]:
#### Optimizer, Loss function and evaluation metric
optimizer = torch.optim.AdamW(params = model.parameters(), lr=LEARNING_RATE)
c_weights = torch.tensor([list(Counter(train_labels).values())[0], list(Counter(train_labels).values())[1]], dtype=torch.float32).to(device)
#Counter.values() helps to see the frequencies of each unique element
c_weights = c_weights/c_weights.sum()
loss_function = torch.nn.CrossEntropyLoss(weight = c_weights)
print("class weights",c_weights)
def calcuate_accu(big_idx, targets):
  n_correct = torch.sum((big_idx==targets) * (targets==1)).item()
  return n_correct
def calculate_labels(targets):
  return torch.sum(targets==1).item()

class weights tensor([0.8870, 0.1130], device='cuda:0')


In [29]:
#### Train one epoch
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    n_predict = 0
    n_ground = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        lengths = data['lengths'].squeeze(1)
        targets = data['targets'].to(device, dtype = torch.long)
        outputs = model(ids, mask,token_type_ids, lengths)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)
        n_predict +=calculate_labels(big_idx)
        n_ground +=calculate_labels(targets)
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        if _%1000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 1000 steps: {loss_step}")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Accuracy Epoch {epoch}: {epoch_accu} and Loss: {epoch_loss}")
    P = n_correct/n_predict if n_predict else 0.0
    R = n_correct/n_ground if n_ground else 0.0
    F1 = (2*P*R)/(P+R) if P+R else 0.0
    print(f"Training P and R and F1 Epoch {epoch}: {P},{R}, {F1}")
    return

#### Valid one epoch
def valid(epoch,model, valid_loader):
    model.eval()
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    n_correct = 0
    n_predict = 0
    n_ground = 0
    with torch.no_grad():
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            lengths = data['lengths'].squeeze(1)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask,token_type_ids, lengths)
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)
            n_predict +=calculate_labels(big_idx)
            n_ground +=calculate_labels(targets)
            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Accuracy Epoch {epoch}: {epoch_accu} and Loss: {epoch_loss}")
    P = n_correct/n_predict if n_predict else 0.0
    R = n_correct/n_ground if n_ground else 0.0
    F1 = (2*P*R)/(P+R) if P+R else 0.0
    print(f"Valiadation  P and R and F1 Epoch {epoch}: {P},{R}, {F1}")
    return epoch_loss



In [33]:
# Run model.
for epoch in range(3):
    train(epoch)
    epoch_loss = valid(epoch,model, valid_loader)
    torch.save(model,f"/content/drive/MyDrive/task-1.pt")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training Loss per 1000 steps: 0.015750136226415634
Training Loss per 1000 steps: 0.09172578943013163
Training Accuracy Epoch 0: 0.0 and Loss: 0.0929053740103699
Training P and R and F1 Epoch 0: 0.0,0.0, 0.0
Validation Accuracy Epoch 0: 0.0 and Loss: 0.09361044676932559
Valiadation  P and R and F1 Epoch 0: 0.0,0.0, 0.0
Training Loss per 1000 steps: 0.17975355684757233
Training Loss per 1000 steps: 0.09304322965059812
Training Accuracy Epoch 1: 0.0 and Loss: 0.0935872724890819
Training P and R and F1 Epoch 1: 0.0,0.0, 0.0
Validation Accuracy Epoch 1: 0.0 and Loss: 0.09437640069387435
Valiadation  P and R and F1 Epoch 1: 0.0,0.0, 0.0
Training Loss per 1000 steps: 0.0134670315310359
Training Loss per 1000 steps: 0.09227980047066163
Training Accuracy Epoch 2: 0.0 and Loss: 0.09176475475391611
Training P and R and F1 Epoch 2: 0.0,0.0, 0.0
Validation Accuracy Epoch 2: 0.0 and Loss: 0.09777844749558508
Valiadation  P and R and F1 Epoch 2: 0.0,0.0, 0.0


In [34]:
!git clone https://github.com/ncg-task/test-data.git "/content/test"

Cloning into '/content/test'...
remote: Enumerating objects: 2508, done.
remote: Total 2508 (delta 0), reused 0 (delta 0), pack-reused 2508
Receiving objects: 100% (2508/2508), 215.28 MiB | 14.99 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Checking out files: 100% (2060/2060), done.


In [45]:
def Articlesandcontributions2(data_dir):
    articles = []
    contributions = []

    for category in os.listdir(data_dir):
        if category != 'README.md' and category != '.git' and category != 'submission.zip':
          article_category = os.path.join(data_dir, category)
          # print(article_category)

          for foldname in sorted(os.listdir(article_category)):
              article_index = os.path.join(article_category, foldname)

              # print(glob.glob(os.path.join(article_index, '*-Stanza-out.txt')))
              # if len(glob.glob(os.path.join(article_index, '*-Stanza-out.txt'))) != 0:
              with open(glob.glob(os.path.join(article_index, '*-Stanza-out.txt'))[0], encoding='utf-8') as f:
                  article = f.read()
                  articles.append(article.lower())

              with open(os.path.join(article_index, 'sentences.txt'), encoding='utf-8') as f:
                  contribution = []
                  for line in f.readlines():
                      article_contribution = int(line.strip())
                      contribution.append(article_contribution)
                  contributions.append(contribution)
              # break
          # break
    return articles, contributions

def SentenceAndLables2(articles, contributions):
    sentences = []
    labels = []
    for i, article in enumerate(articles):
        contribution = contributions[i]

        sents = article.split('\n')[0:-1]
        rand_sents=random.sample(sents,30)
        # sents = article.split('\n')
        rand_sents=random.sample(rand_sents,10)
        for row, sent in enumerate(rand_sents):
            sentences.append(sent)
            
            if (row + 1) in contribution:
                labels.append(1)
            else:
                labels.append(0)
    # print(sentences)
    # print(labels)
    return sentences, labels

In [50]:
test_input_dir='/content/test'

test_articles, test_contributions = Articlesandcontributions2(test_input_dir)

test_sentences, test_labels = SentenceAndLables2(test_articles, test_contributions)
test_set = Triage(test_sentences,test_labels, tokenizer, MAX_LEN)
test_params = {'batch_size': 8,'shuffle': True,'num_workers': 0}
testing_loader = DataLoader(test_set, **test_params)


In [ ]:
for _,data in enumerate(testing_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        lengths = data['lengths'].squeeze(1)
        targets = data['targets'].to(device, dtype = torch.long)
        outputs = model(ids, mask,token_type_ids, lengths)
        
        